In [93]:
import pandas as pd
import numpy as np
import math

In [94]:
data = pd.read_csv('fifa21_train.csv')
data.head().T

,0,1,2,3,4
ID,184383,188044,184431,233796,234799
Name,A. Pasche,Alan Carvalho,S. Giovinco,J. Evans,Y. Demoncy
Age,26,30,33,22,23
Nationality,Switzerland,China PR,Italy,Wales,France
Club,FC Lausanne-Sport,Beijing Sinobo Guoan FC,Al Hilal,Swansea City,US Orléans Loiret Football
...,...,...,...,...,...
CB,54+1,48+2,41+2,58+2,61+2
RCB,54+1,48+2,41+2,58+2,61+2
RB,58+1,53+2,53+2,57+2,63+2
GK,15+1,18+2,12+2,14+2,15+2


In [95]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(data)

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,€525K,€4K,€801K,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,0,"Jan 16, 2015","Dec 31, 2020",€8.5M,€23K,€0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11699,146717,Anderson Silva,26,Brazil,Barnsley,CM,NaN,Barnsley 2010,"6'2""",179lbs,Right,7,NaN,NaN,€0,€0,€0,2010,254,64,66,51,73,NaN,284,74,NaN,63,72,75,198,71,72,NaN,55,NaN,258,61,NaN,71,64,62,241,64,61.0,63.0,NaN,53,NaN,113,55,58,NaN,81,9,24,72,24,24,1429,400,3 ★,1★,NaN,NaN,3 ★,72,63,70,72,57,66,3,68+0,68+0,68+0,71+0,68+0,68+0,68+0,71+0,72+0,72+0,72+0,71+0,71+0,71+0,71+0,71+0,66+0,68+0,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0,68
11700,236507,T. Conechny,22,Argentina,Portland Timbers,CAM,ST CAM LM,Portland Timbers 2019 ~ 2020,"5'7""",143lbs,Left,9,"Jul 22, 2019",NaN,€700K,€2K,€1.2M,2019 ~ 2020,289,43,64,65,64,53.0,267,64,41.0,43,55,64,391,81,78,87.0,61,84.0,312,62,85.0,65,42,58,244,34,20.0,66.0,63.0,61,56.0,56,20,18,18.0,48,10,8,14,8,8,1607,336,3 ★,3★,High,Low,1 ★,79,62,56,67,24,48,8,64+2,64+2,64+2,65+0,65+0,65+0,65+0,65+0,65+2,65+2,65+2,63+2,58+2,58+2,58+2,63+2,46+2,43+2,43+2,43+2,46+2,44+2,36+2,36+2,36+2,44+2,15+2,64


In [96]:
data = data.drop_duplicates()
data.shape

(11701, 101)

Drop: ID, Name, Team & Contract
oh: Nationality, Club, BP, *Position*, foot, 
label enc: A/W, D/W
doubts: Joined, *Loan Date End*, *Contract*, 

In [97]:
data = data.drop(['ID', 'Name', 'Team & Contract'], axis=1)
data.head()

,Age,Nationality,Club,BP,Position,Height,Weight,foot,Growth,Joined,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,"5'9""",161lbs,Right,1,"Jul 1, 2015",...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"6'0""",159lbs,Right,0,"Jan 16, 2015",...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,33,Italy,Al Hilal,CAM,CAM CF,"5'4""",134lbs,Right,0,"Jan 31, 2019",...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,22,Wales,Swansea City,CDM,CDM CM,"5'10""",152lbs,Right,13,"Jul 1, 2016",...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,23,France,US Orléans Loiret Football,CDM,CDM CM,"5'11""",150lbs,Right,8,"Jul 1, 2018",...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [98]:
data.columns = [colname.lower().replace(' ','_') for colname in data.columns]
data.columns

Index(['age', 'nationality', 'club', 'bp', 'position', 'height', 'weight',
       'foot', 'growth', 'joined', 'loan_date_end', 'value', 'wage',
       'release_clause', 'contract', 'attacking', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'skill', 'dribbling',
       'curve', 'fk_accuracy', 'long_passing', 'ball_control', 'movement',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'power', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'mentality', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'composure', 'defending', 'marking', 'standing_tackle',
       'sliding_tackle', 'goalkeeping', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes', 'total_stats',
       'base_stats', 'w/f', 'sm', 'a/w', 'd/w', 'ir', 'pac', 'sho', 'pas',
       'dri', 'def', 'phy', 'hits', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf',
       'rw', 'lam', 'cam', 'ram', '

In [99]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(data)

,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,€525K,€4K,€801K,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"6'0""",159lbs,Right,0,"Jan 16, 2015","Dec 31, 2020",€8.5M,€23K,€0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11699,26,Brazil,Barnsley,CM,NaN,"6'2""",179lbs,Right,7,NaN,NaN,€0,€0,€0,2010,254,64,66,51,73,NaN,284,74,NaN,63,72,75,198,71,72,NaN,55,NaN,258,61,NaN,71,64,62,241,64,61.0,63.0,NaN,53,NaN,113,55,58,NaN,81,9,24,72,24,24,1429,400,3 ★,1★,NaN,NaN,3 ★,72,63,70,72,57,66,3,68+0,68+0,68+0,71+0,68+0,68+0,68+0,71+0,72+0,72+0,72+0,71+0,71+0,71+0,71+0,71+0,66+0,68+0,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0,68
11700,22,Argentina,Portland Timbers,CAM,ST CAM LM,"5'7""",143lbs,Left,9,"Jul 22, 2019",NaN,€700K,€2K,€1.2M,2019 ~ 2020,289,43,64,65,64,53.0,267,64,41.0,43,55,64,391,81,78,87.0,61,84.0,312,62,85.0,65,42,58,244,34,20.0,66.0,63.0,61,56.0,56,20,18,18.0,48,10,8,14,8,8,1607,336,3 ★,3★,High,Low,1 ★,79,62,56,67,24,48,8,64+2,64+2,64+2,65+0,65+0,65+0,65+0,65+0,65+2,65+2,65+2,63+2,58+2,58+2,58+2,63+2,46+2,43+2,43+2,43+2,46+2,44+2,36+2,36+2,36+2,44+2,15+2,64


In [100]:
data['weight'] = pd.to_numeric(data['weight'].map(lambda x: x.rstrip('lbs')))
data['weight'] 

0        161
1        159
2        134
3        152
4        150
        ... 
11696    168
11697    174
11698    143
11699    179
11700    143
Name: weight, Length: 11701, dtype: int64

In [101]:
def clean_money_values(column):
    #Works the columns with format '€xK' or '€xM'
    #Return the value thousands of euros
    for i in range(column.size):
        if column[i].find('K') > 0:
            column[i] = float(column[i].lstrip('€').rstrip('K'))
        elif column[i].find('M') > 0:
            column[i] = float(column[i].lstrip('€').rstrip('M')) * 1000
        else:
            column[i] = float(column[i].lstrip('€'))
    return column

In [102]:
data['value'] = clean_money_values(data['value'])
data['wage'] = clean_money_values(data['wage'])
data['release_clause'] = clean_money_values(data['release_clause'])

C:\Users\carlo\AppData\Local\Temp/ipykernel_5940/2010045874.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[i] = float(column[i].lstrip('€').rstrip('K'))
C:\Users\carlo\AppData\Local\Temp/ipykernel_5940/2010045874.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[i] = float(column[i].lstrip('€').rstrip('M')) * 1000


In [103]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(data)

,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,"5'9""",161,Right,1,"Jul 1, 2015",NaN,525.0,4.0,801.0,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"6'0""",159,Right,0,"Jan 16, 2015","Dec 31, 2020",8500.0,23.0,€0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11699,26,Brazil,Barnsley,CM,NaN,"6'2""",179,Right,7,NaN,NaN,€0,€0,€0,2010,254,64,66,51,73,NaN,284,74,NaN,63,72,75,198,71,72,NaN,55,NaN,258,61,NaN,71,64,62,241,64,61.0,63.0,NaN,53,NaN,113,55,58,NaN,81,9,24,72,24,24,1429,400,3 ★,1★,NaN,NaN,3 ★,72,63,70,72,57,66,3,68+0,68+0,68+0,71+0,68+0,68+0,68+0,71+0,72+0,72+0,72+0,71+0,71+0,71+0,71+0,71+0,66+0,68+0,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0,68
11700,22,Argentina,Portland Timbers,CAM,ST CAM LM,"5'7""",143,Left,9,"Jul 22, 2019",NaN,700.0,2.0,1200.0,2019 ~ 2020,289,43,64,65,64,53.0,267,64,41.0,43,55,64,391,81,78,87.0,61,84.0,312,62,85.0,65,42,58,244,34,20.0,66.0,63.0,61,56.0,56,20,18,18.0,48,10,8,14,8,8,1607,336,3 ★,3★,High,Low,1 ★,79,62,56,67,24,48,8,64+2,64+2,64+2,65+0,65+0,65+0,65+0,65+0,65+2,65+2,65+2,63+2,58+2,58+2,58+2,63+2,46+2,43+2,43+2,43+2,46+2,44+2,36+2,36+2,36+2,44+2,15+2,64


In [104]:
data['height'].unique() 

array(['5\'9"', '6\'0"', '5\'4"', '5\'10"', '5\'11"', '6\'4"', '6\'2"',
       '5\'5"', '5\'8"', '6\'3"', '6\'1"', '5\'6"', '6\'5"', '5\'7"',
       '6\'6"', '5\'3"', '6\'7"', '6\'8"', '5\'1"', '5\'2"', '6\'9"'],
      dtype=object)

In [105]:
for i in range(data['height'].size):
    h = data['height'][i].rstrip('"').split("'")
    data['height'][i] = round(((float(h[0])*12) + float(h[1])) * 2.54, 1)
data['height']   

C:\Users\carlo\AppData\Local\Temp/ipykernel_5940/1733765111.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['height'][i] = round(((float(h[0])*12) + float(h[1])) * 2.54, 1)


0        175.3
1        182.9
2        162.6
3        177.8
4        180.3
         ...  
11696    185.4
11697    185.4
11698    167.6
11699    188.0
11700    170.2
Name: height, Length: 11701, dtype: object

In [106]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(data)

,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,175.3,161,Right,1,"Jul 1, 2015",NaN,525.0,4.0,801.0,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,182.9,159,Right,0,"Jan 16, 2015","Dec 31, 2020",8500.0,23.0,€0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11699,26,Brazil,Barnsley,CM,NaN,188.0,179,Right,7,NaN,NaN,€0,€0,€0,2010,254,64,66,51,73,NaN,284,74,NaN,63,72,75,198,71,72,NaN,55,NaN,258,61,NaN,71,64,62,241,64,61.0,63.0,NaN,53,NaN,113,55,58,NaN,81,9,24,72,24,24,1429,400,3 ★,1★,NaN,NaN,3 ★,72,63,70,72,57,66,3,68+0,68+0,68+0,71+0,68+0,68+0,68+0,71+0,72+0,72+0,72+0,71+0,71+0,71+0,71+0,71+0,66+0,68+0,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0,68
11700,22,Argentina,Portland Timbers,CAM,ST CAM LM,170.2,143,Left,9,"Jul 22, 2019",NaN,700.0,2.0,1200.0,2019 ~ 2020,289,43,64,65,64,53.0,267,64,41.0,43,55,64,391,81,78,87.0,61,84.0,312,62,85.0,65,42,58,244,34,20.0,66.0,63.0,61,56.0,56,20,18,18.0,48,10,8,14,8,8,1607,336,3 ★,3★,High,Low,1 ★,79,62,56,67,24,48,8,64+2,64+2,64+2,65+0,65+0,65+0,65+0,65+0,65+2,65+2,65+2,63+2,58+2,58+2,58+2,63+2,46+2,43+2,43+2,43+2,46+2,44+2,36+2,36+2,36+2,44+2,15+2,64
